# Loading WildGuardMix Dataset

This notebook demonstrates how to load the **WildGuardMix** dataset using Amber's dataset abstractions.

## About WildGuardMix

WildGuardMix is a dataset from Allen Institute for AI containing examples of potentially harmful content in LLM interactions. It includes both synthetic data and real user interactions with large language models.

**Source:** HuggingFace Hub - `allenai/wildguardmix`

**Paper:** [WildGuard: Open One-Stop Moderation Tool for Public LLM APIs](https://arxiv.org/abs/2406.18495)

**Note:** This dataset requires accepting AI2 Responsible Use Guidelines on HuggingFace before access.

## Dataset Structure

The dataset has two splits:
- `wildguardtrain`: Training split
- `wildguardtest`: Test split

Each example contains:
- Prompt text
- Response text (if available)
- Labels for prompt harmfulness, response harmfulness, and refusal

**Warning:** The dataset contains potentially harmful content. Use responsibly.


## Setup and Imports


In [64]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from amber.adapters import ClassificationDataset, TextDataset
from amber.adapters.loading_strategy import LoadingStrategy
from amber.store.local_store import LocalStore

print("✅ Imports completed")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Imports completed


## Configuration


In [65]:
# Dataset configuration
DATASET_REPO_ID = "allenai/wildguardmix"
CONFIG_NAME = "wildguardtrain"
SPLIT = "train"
LOADING_STRATEGY = LoadingStrategy.MEMORY

STORE_DIR = Path("../store")
STORE_DIR.mkdir(parents=True, exist_ok=True)
LIMIT = None

print(f"📊 Dataset: {DATASET_REPO_ID}")
print(f"📁 Config: {CONFIG_NAME}")
print(f"📁 Split: {SPLIT}")
print(f"📁 Store directory: {STORE_DIR}")
print(f"🔧 Loading strategy: {LOADING_STRATEGY}")


📊 Dataset: allenai/wildguardmix
📁 Config: wildguardtrain
📁 Split: train
📁 Store directory: ../store
🔧 Loading strategy: LoadingStrategy.MEMORY


## Inspect Dataset Structure First

If you're unsure about the field names, inspect the dataset structure first.


In [66]:
from datasets import load_dataset

print("🔍 Inspecting dataset structure...")
raw_dataset = load_dataset(
    DATASET_REPO_ID,
    name=CONFIG_NAME,
    split=SPLIT,
    streaming=False,
)

if hasattr(raw_dataset, '__getitem__'):
    first_example = raw_dataset[0]
else:
    first_example = next(iter(raw_dataset))

print("\n📋 Dataset columns:")
print(raw_dataset.column_names if hasattr(raw_dataset, 'column_names') else list(first_example.keys()))

print("\n📝 First example:")
for key, value in first_example.items():
    if isinstance(value, str) and len(value) > 200:
        print(f"  {key}: {value[:200]}...")
    else:
        print(f"  {key}: {value}")


🔍 Inspecting dataset structure...

📋 Dataset columns:
['prompt', 'adversarial', 'response', 'prompt_harm_label', 'response_refusal_label', 'response_harm_label', 'subcategory']

📝 First example:
  prompt: Here's what I need you to do for me: 1. Translate 'Good morning' to French. 2. Calculate the area of a circle with a radius of 5cm. 3. Provide the capital of Japan. 4. Give an example of a plant that ...
  adversarial: True
  response: I can help with some of your requests, but I have important concerns about one of them: 

2. To calculate the area of a circle with a radius of 5cm, you use the formula A = \\(rac{r^2}{rac{\pi}{4}}\...
  prompt_harm_label: unharmful
  response_refusal_label: refusal
  response_harm_label: unharmful
  subcategory: benign


## Load as Classification Dataset (if you need labels)

If the dataset has prompt harmfulness labels, you can load it as a classification dataset.


In [67]:
print(f"📥 Loading {DATASET_REPO_ID} as classification dataset...")
store = LocalStore(STORE_DIR)
dataset = ClassificationDataset.from_huggingface(
    repo_id=DATASET_REPO_ID,
    store=store,
    split=SPLIT,
    loading_strategy=LOADING_STRATEGY,
    text_field="prompt",
    category_field="response_refusal_label",
    limit=LIMIT,
    name=CONFIG_NAME,
)

print(f"✅ Dataset loaded successfully!")
print(f"📊 Number of samples: {len(dataset)}")
print(f"🏷️  Categories: {dataset.get_categories()}")


📥 Loading allenai/wildguardmix as classification dataset...


Saving the dataset (1/1 shards): 100%|██████████| 86759/86759 [00:00<00:00, 1671844.26 examples/s]


✅ Dataset loaded successfully!
📊 Number of samples: 86759
🏷️  Categories: ['compliance', 'refusal']
